In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()
app_token = os.getenv('RAGAS_APP_TOKEN')

In [2]:
import ast
import pandas as pd
from ragas import SingleTurnSample, EvaluationDataset

def load_dataset() -> EvaluationDataset:
    df = pd.read_csv('dataset.csv')

    # Convert the string representation of lists to actual Python lists
    df['retrieved_contexts'] = df['retrieved_contexts'].apply(ast.literal_eval)

    samples = []
    for i in range(len(df)):
        sample = SingleTurnSample(
            user_input = df['user_input'].iloc[i],
            retrieved_contexts = df['retrieved_contexts'].iloc[i],
            response = df['response'].iloc[i],
            reference = df['reference'].iloc[i]
        )
        samples.append(sample)

    eval_dataset = EvaluationDataset(samples)
    return eval_dataset

eval_dataset = load_dataset()

/home/p3tr0vv/Desktop/Evaluation-Approaches-for-Retrieval-Augmented-Generation-RAG-/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://localhost:11434"
)

chat_model = OllamaLLM(
    model="llama3.1",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_ctx=24000
)

In [4]:
from ragas.cache import DiskCacheBackend
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

cacher = DiskCacheBackend(".cache")

llm = LangchainLLMWrapper(
    langchain_llm=chat_model,
    cache=cacher
)
embeddings = LangchainEmbeddingsWrapper(
    embeddings=embeddings_model,
    cache=cacher
)

In [5]:
from ragas.metrics import Faithfulness
from ragas.evaluation import evaluate
from ragas.run_config import RunConfig

run_config = RunConfig(
    timeout = 3600, # One hour
    max_wait = 30,
    log_tenacity = True
)

faithfulness = Faithfulness()

result = evaluate(
    dataset=eval_dataset,
    metrics=[faithfulness],
    llm=llm,
    embeddings=embeddings,
    run_config=run_config
)

Evaluating: 100%|██████████| 25/25 [28:31<00:00, 68.47s/it]  


In [ ]:
result_df = result.to_pandas()
result_df.to_csv('eval_results/faithfulness.csv', index=False)

In [10]:
result.upload()

[2025-03-22 12:46:55 - (2025-03-22 11:46:55 UTC)] [ERROR] [ragas.utils] [RagasID: a-e1d425efa8cd4f2285c3c46fcc2e0726, App-Version: 0.2.14] [API_ERROR] Request failed. Status Code: 500, URL: https://api.ragas.io/api/v1/alignment/evaluation, Error Message: 
API Message: An internal server error occured


UploadException: Request failed: 
API Message: An internal server error occured